<a href="https://colab.research.google.com/github/demoleiwang/SDSC_Bert_Seminar/blob/master/03_Sentiment_Analysis_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!gdown --id '1vP1lVYFGTLGHjvST3kSH5pxowd_4DcAe' --output IMDB_Dataset.csv
!gdown --id '1dzAwSDVcoo0Z39gOKVGDECFoBYbyFhxf' --output tut6-model.pt
! pip install transformers
! ls

Downloading...
From: https://drive.google.com/uc?id=1vP1lVYFGTLGHjvST3kSH5pxowd_4DcAe
To: /content/IMDB_Dataset.csv
66.2MB [00:00, 129MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1dzAwSDVcoo0Z39gOKVGDECFoBYbyFhxf
To: /content/tut6-model.pt
449MB [00:05, 81.2MB/s]
     |████████████████████████████████| 778kB 4.8MB/s 
     |████████████████████████████████| 1.1MB 23.9MB/s 
     |████████████████████████████████| 3.0MB 42.0MB/s 
     |████████████████████████████████| 890kB 35.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=63b31af550764f1211a1d62914f15d80aea721a1ae2796711d143bcef50e8859
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
IMDB_Dataset.csv  sample_data  tut6-model.pt


In [2]:
from torchtext import data
from torchtext import datasets

import torch
from transformers import BertTokenizer, BertModel

The most parts of the following code are similar to 03-sentiment_analysis.ipynb. 

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert = BertModel.from_pretrained('bert-base-uncased')

max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']
print(max_input_length)

def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_input_length-2]
    return tokens

REVIEW = data.Field(batch_first=True,
                    use_vocab=False,
                    tokenize = tokenize_and_cut,
                    preprocessing = tokenizer.convert_tokens_to_ids,
                    init_token = tokenizer.cls_token_id,
                    eos_token = tokenizer.sep_token_id,
                    pad_token = tokenizer.pad_token_id,
                    unk_token = tokenizer.unk_token_id
                   )
SENTIMENT = data.LabelField(dtype = torch.float)

fields = {'review': ('r', REVIEW), 'sentiment': ('s', SENTIMENT)}

IMDB_data = data.TabularDataset(
    path = './IMDB_Dataset.csv',
    format = 'csv',
    fields = fields)

train_data, valid_data, test_data = IMDB_data.split(split_ratio=[0.8, 0.1, 0.1])

len(train_data), len(valid_data), len(test_data)

# REVIEW.build_vocab(train_data)
SENTIMENT.build_vocab(train_data)


512


In [4]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print ("device:", device)

BATCH_SIZE = 16

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    sort = False, #don't sort test/validation data
    batch_size=BATCH_SIZE,
    device=device)

import torch.nn as nn


class BERTGRUSentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):

        super().__init__()

        self.bert = bert

        embedding_dim = bert.config.to_dict()['hidden_size']

        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers=n_layers,
                          bidirectional=bidirectional,
                          batch_first=True,
                          dropout=0 if n_layers < 2 else dropout)

        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, text):

        # text = [batch size, sent len]

        with torch.no_grad():
            embedded = self.bert(text)[0]

        # embedded = [batch size, sent len, emb dim]

        _, hidden = self.rnn(embedded)

        # hidden = [n layers * n directions, batch size, emb dim]

        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1))
        else:
            hidden = self.dropout(hidden[-1, :, :])

        # hidden = [batch size, hid dim]

        output = self.out(hidden)

        # output = [batch size, out dim]

        return output

HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25

model = BERTGRUSentiment(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)

device: cuda


Load the model and then evaluate it in test dataset.

In [5]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():
        for batch in iterator:
            input_tensor = batch.r  # .transpose(1,0)
            ground_y = batch.s.squeeze(0)
            #         print (input_tensor.size())
            #         print (ground_y.size())

            predictions = model(input_tensor).squeeze(1)

            loss = (predictions, ground_y)

            acc = binary_accuracy(predictions, ground_y)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [9]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division
    acc = correct.sum() / len(correct)
    return acc

In [6]:
model.load_state_dict(torch.load('./tut6-model.pt'))

<All keys matched successfully>

In [10]:

criterion = nn.BCEWithLogitsLoss()
model.to(device)
criterion = criterion.to(device)

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.148 | Test Acc: 94.43%


Show one positive and one negative examples

In [14]:
def predict_sentiment(model, tokenizer, sentence):
    model.eval()
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_input_length-2]
    indexed = [tokenizer.cls_token_id] + tokenizer.convert_tokens_to_ids(tokens) + [tokenizer.sep_token_id]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [15]:
predict_sentiment(model, tokenizer, "This movie is terrible!")

0.006466880440711975

In [16]:
predict_sentiment(model, tokenizer, "This movie is so great!")

0.986177921295166